In [3]:
!pip install beautifulsoup4

## Part 1 - Build a dataframe of Toronto Neighborhood Data

In [1]:
# import required library packages
import numpy as np 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url).text
soup = BeautifulSoup(response, "lxml")
table = soup.find_all('table')[0]

In [3]:
# data cleaning
table_contents=[]
table=soup.find('table')
for row in table.find_all('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [4]:
df.shape

(103, 3)

In [15]:
print('The dataframe has {} boroughs.'.format(
        len(df['Borough'].unique())
    )
)

The dataframe has 15 boroughs.


In [12]:
df.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## Part 2 - get the latitude and the longitude coordinates of each neighborhood

In [8]:
!pip install geocoder

  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=e2205b4c5b4f57139b1031ba55ceba9be86994db24ed3efb6afc08d6d8e23670
  Stored in directory: c:\users\xinyue.bai\appdata\local\pip\cache\wheels\6e\9c\ed\4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built future


In [ ]:
# import geocoder
# this method takes a very long time, so i will just read the csv file provided instead
import geocoder 
df['Latitude'] = 0
df['Longitude'] = 0
for postal_code in df['PostalCode']:
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    print(lat_lng_coords)
    
    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]
    df[df['PostalCode'] == postal_code]['Latitude'] = lat_lng_coords[0]
    df[df['PostalCode'] == postal_code]['Longitude'] = lat_lng_coords[1]

In [8]:
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')
geospatial_data.head()
geospatial_data['PostalCode'] = geospatial_data['Postal Code']
geospatial_data.drop('Postal Code', axis=1, inplace=True)
geospatial_data.head()

,Latitude,Longitude,PostalCode
0,43.806686,-79.194353,M1B
1,43.784535,-79.160497,M1C
2,43.763573,-79.188711,M1E
3,43.770992,-79.216917,M1G
4,43.773136,-79.239476,M1H


In [11]:
final_df = pd.merge(df, geospatial_data, on='PostalCode')
final_df.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3 - cluster the neighborhoods based on latitude and longitude

In [27]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [28]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [30]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(final_df[['Latitude', 'Longitude']])

# check cluster labels generated for each row in the dataframe
final_df['Cluster'] = kmeans.labels_

In [32]:
final_df.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,4
1,M4A,North York,Victoria Village,43.725882,-79.315572,4
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,2
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,1
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0
7,M3B,North York,Don Mills North,43.745906,-79.352188,4
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,4
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2


In [34]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
colors = ['red', 'yellow', 'blue', 'green', 'grey']

# add markers to map
for lat, lng, borough, neighborhood, cluster in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighborhood'], final_df['Cluster']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto